In [1]:
import Experiments_Module as exp

In [2]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
import h5py
import scipy.io as sio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
get_ipython().magic('matplotlib inline')

from pylab import rcParams
rcParams['figure.figsize'] = 14, 12 
get_ipython().magic("config InlineBackend.figure_format = 'retina'  # делаем картинки красивее")

pd.options.display.max_rows = 20

In [3]:
exp.rnd = np.random.RandomState(seed=317)

In [4]:
results = pd.DataFrame()

# IONOSPHERE

In [6]:
DF = pd.read_csv("ionosphere.csv", sep=';', header=None)
ans = (DF[34] == 'b')
train_ans, ans = ans[:100], ans[100:]
del DF[34]

DF, DF_test = DF[:100], DF[100:]
DF

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,...,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.20332,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,0.57528,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,1.00000,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,0.03286,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697
5,1,0,0.02337,-0.00592,-0.09924,-0.11949,-0.00763,-0.11824,0.14706,0.06637,...,0.03513,-0.01535,-0.03240,0.09223,-0.07859,0.00732,0.00000,0.00000,-0.00039,0.12011
6,1,0,0.97588,-0.10602,0.94601,-0.20800,0.92806,-0.28350,0.85996,-0.27342,...,0.22792,-0.81634,0.13659,-0.82510,0.04606,-0.82395,-0.04262,-0.81318,-0.13832,-0.80975
7,0,0,0.00000,0.00000,0.00000,0.00000,1.00000,-1.00000,0.00000,0.00000,...,1.00000,1.00000,1.00000,1.00000,0.00000,0.00000,1.00000,1.00000,0.00000,0.00000
8,1,0,0.96355,-0.07198,1.00000,-0.14333,1.00000,-0.21313,1.00000,-0.36174,...,0.85106,-0.65440,0.57577,-0.69712,0.25435,-0.63919,0.45114,-0.72779,0.38895,-0.73420
9,1,0,-0.01864,-0.08459,0.00000,0.00000,0.00000,0.00000,0.11470,-0.26810,...,-0.14803,-0.01326,0.20645,-0.02294,0.00000,0.00000,0.16595,0.24086,-0.08208,0.38065


In [7]:
scal = StandardScaler().fit(DF)
DF = scal.transform(DF)
DF_test = scal.transform(DF_test)
train_ans = train_ans.as_matrix()
ans = ans.as_matrix()

RES = {}

In [7]:
RES["IsolationForest"] = exp.Solve(IsolationForest(100, max_samples=1.0), DF, train_ans, DF_test, ans)
RES["IsolationForest S"] = exp.Solve(IsolationForest(100, max_samples=250), DF, train_ans, DF_test, ans)
RES["IsolationForest RB"] = exp.Solve(exp.Ensembler(exp.Rotated(IsolationForest(1, max_samples=1.0))), DF, train_ans, DF_test, ans)
RES["IsolationForest RB-S"] = exp.Solve(exp.Ensembler(exp.Rotated(IsolationForest(1, max_samples=250))), DF, train_ans, DF_test, ans)
RES["IsolationForest RB-RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(IsolationForest(1, max_samples=1.0)))), DF, train_ans, DF_test, ans)
RES["IsolationForest RB-RFW-S"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(IsolationForest(1, max_samples=250)))), DF, train_ans, DF_test, ans)

C:\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:184: UserWarning: max_samples (250) is greater than the total number of samples (100). max_samples will be set to n_samples for estimation.
  % (self.max_samples, n_samples))
C:\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:184: UserWarning: max_samples (250) is greater than the total number of samples (100). max_samples will be set to n_samples for estimation.
  % (self.max_samples, n_samples))
C:\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:184: UserWarning: max_samples (250) is greater than the total number of samples (100). max_samples will be set to n_samples for estimation.
  % (self.max_samples, n_samples))
C:\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:184: UserWarning: max_samples (250) is greater than the total number of samples (100). max_samples will be set to n_samples for estimation.
  % (self.max_samples, n_samples))
C:\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:184: 

In [8]:
RES["LOF"] = exp.Solve(exp.LOF(metric="chebyshev"), DF, train_ans, DF_test, ans)
RES["LOF S"] = exp.Solve(exp.Sampler(exp.LOF(metric="chebyshev"), 100), DF, train_ans, DF_test, ans)
RES["LOF RB"] = exp.Solve(exp.Ensembler(exp.Rotated(exp.LOF(metric="chebyshev")), 100), DF, train_ans, DF_test, ans)
RES["LOF RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.LOF(metric="chebyshev")), 100), DF, train_ans, DF_test, ans)
RES["LOF RB-S"] = exp.Solve(exp.Sampler(exp.Rotated(exp.LOF(metric="chebyshev")), 100), DF, train_ans, DF_test, ans)
RES["LOF RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.LOF(metric="chebyshev")), 100), DF, train_ans, DF_test, ans)
RES["LOF RB-RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(exp.LOF(metric="chebyshev"))), 100), DF, train_ans, DF_test, ans)
RES["LOF RB-RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.Rotated(exp.LOF(metric="chebyshev"))), 100), DF, train_ans, DF_test, ans)

RES["Polynom"] = exp.Solve(exp.PolynomSolver(metric="chebyshev"), DF, train_ans, DF_test, ans)
RES["Polynom S"] = exp.Solve(exp.Sampler(exp.PolynomSolver(metric="chebyshev"), 100), DF, train_ans, DF_test, ans)
RES["Polynom RB"] = exp.Solve(exp.Ensembler(exp.Rotated(exp.PolynomSolver(metric="chebyshev")), 100), DF, train_ans, DF_test, ans)
RES["Polynom RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.PolynomSolver(metric="chebyshev")), 100), DF, train_ans, DF_test, ans)
RES["Polynom RB-S"] = exp.Solve(exp.Sampler(exp.Rotated(exp.PolynomSolver(metric="chebyshev")), 100), DF, train_ans, DF_test, ans)
RES["Polynom RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.PolynomSolver(metric="chebyshev")), 100), DF, train_ans, DF_test, ans)
RES["Polynom RB-RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(exp.PolynomSolver(metric="chebyshev"))), 100), DF, train_ans, DF_test, ans)
RES["Polynom RB-RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.Rotated(exp.PolynomSolver(metric="chebyshev"))), 100), DF, train_ans, DF_test, ans)

RES["IterativeEnsemble"] = exp.Solve(exp.IterativeEnsemble(metric="chebyshev"), DF, train_ans, DF_test, ans)

C:\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:184: UserWarning: max_samples (250) is greater than the total number of samples (100). max_samples will be set to n_samples for estimation.
  % (self.max_samples, n_samples))


In [9]:
results["Ionosphere (train)"] = pd.Series({key: item[0] for key, item in RES.items()})
results["Ionosphere (test)"] = pd.Series({key: item[1] for key, item in RES.items()})
results.to_csv("FINAL_RESULTS.csv")
results

,Ionosphere (train),Ionosphere (test)
IsolationForest,0.923169,0.812883
IsolationForest RB,0.929172,0.805792
IsolationForest RB-RFW,0.932373,0.792432
IsolationForest RB-RFW-S,0.937175,0.811390
IsolationForest RB-S,0.926771,0.808031
IsolationForest S,0.927171,0.794298
IterativeEnsemble,0.910364,0.828034
LOF,0.960384,0.874832
LOF RB,0.903962,0.810942
LOF RB-RFW,0.919168,0.825123


# ARRHYTHMIA

In [70]:
DF = pd.read_csv("arrhythmia.csv", sep=',', header=None)
for c in DF.columns:
    DF[c][DF[c].astype(str) == '?'] = -1
    DF[c] = DF[c].astype(float)
ans = ((DF[279] == 3) | (DF[279] == 4) | (DF[279] == 5) | (DF[279] == 7) | (DF[279] == 8)
       | (DF[279] == 9) | (DF[279] == 14) | (DF[279] == 15))
train_ans, ans = ans[:300], ans[300:]
del DF[279]

DF, DF_test = DF[:300], DF[300:]
DF

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,0,1,2,3,4,5,6,7,8,9,...,269,270,271,272,273,274,275,276,277,278
0,75.0,0.0,190.0,80.0,91.0,193.0,371.0,174.0,121.0,-16.0,...,-0.3,0.0,9.0,-0.9,0.0,0.0,0.9,2.9,23.3,49.4
1,56.0,1.0,165.0,64.0,81.0,174.0,401.0,149.0,39.0,25.0,...,-0.5,0.0,8.5,0.0,0.0,0.0,0.2,2.1,20.4,38.8
2,54.0,0.0,172.0,95.0,138.0,163.0,386.0,185.0,102.0,96.0,...,0.9,0.0,9.5,-2.4,0.0,0.0,0.3,3.4,12.3,49.0
3,55.0,0.0,175.0,94.0,100.0,202.0,380.0,179.0,143.0,28.0,...,0.1,0.0,12.2,-2.2,0.0,0.0,0.4,2.6,34.6,61.6
4,75.0,0.0,190.0,80.0,88.0,181.0,360.0,177.0,103.0,-16.0,...,-0.4,0.0,13.1,-3.6,0.0,0.0,-0.1,3.9,25.4,62.8
5,13.0,0.0,169.0,51.0,100.0,167.0,321.0,174.0,91.0,107.0,...,0.0,-0.6,12.2,-2.8,0.0,0.0,0.9,2.2,13.5,31.1
6,40.0,1.0,160.0,52.0,77.0,129.0,377.0,133.0,77.0,77.0,...,-0.4,0.0,6.5,0.0,0.0,0.0,0.4,1.0,14.3,20.5
7,49.0,1.0,162.0,54.0,78.0,0.0,376.0,157.0,70.0,67.0,...,-0.3,0.0,8.2,-1.9,0.0,0.0,0.1,0.5,15.8,19.8
8,44.0,0.0,168.0,56.0,84.0,118.0,354.0,160.0,63.0,61.0,...,0.1,0.0,7.0,-1.3,0.0,0.0,0.6,2.1,12.5,30.9
9,50.0,1.0,167.0,67.0,89.0,130.0,383.0,156.0,73.0,85.0,...,-0.5,-0.6,10.8,-1.7,0.0,0.0,0.8,0.9,20.1,25.1


In [73]:
len(DF.columns)

279

In [12]:
scal = StandardScaler().fit(DF)
DF = scal.transform(DF)
DF_test = scal.transform(DF_test)
train_ans = train_ans.as_matrix()
ans = ans.as_matrix()

RES = {}

In [12]:
RES["IsolationForest"] = exp.Solve(IsolationForest(100, max_samples=1.0), DF, train_ans, DF_test, ans)
RES["IsolationForest S"] = exp.Solve(IsolationForest(100, max_samples=250), DF, train_ans, DF_test, ans)
RES["IsolationForest RB"] = exp.Solve(exp.Ensembler(exp.Rotated(IsolationForest(1, max_samples=1.0))), DF, train_ans, DF_test, ans)
RES["IsolationForest RB-S"] = exp.Solve(exp.Ensembler(exp.Rotated(IsolationForest(1, max_samples=250))), DF, train_ans, DF_test, ans)
RES["IsolationForest RB-RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(IsolationForest(1, max_samples=1.0)))), DF, train_ans, DF_test, ans)
RES["IsolationForest RB-RFW-S"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(IsolationForest(1, max_samples=250)))), DF, train_ans, DF_test, ans)

In [13]:
RES["LOF"] = exp.Solve(exp.LOF(metric="manhattan"), DF, train_ans, DF_test, ans)
RES["LOF S"] = exp.Solve(exp.Sampler(exp.LOF(metric="manhattan"), 100), DF, train_ans, DF_test, ans)
RES["LOF RB"] = exp.Solve(exp.Ensembler(exp.Rotated(exp.LOF(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["LOF RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.LOF(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["LOF RB-S"] = exp.Solve(exp.Sampler(exp.Rotated(exp.LOF(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["LOF RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.LOF(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["LOF RB-RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(exp.LOF(metric="manhattan"))), 100), DF, train_ans, DF_test, ans)
RES["LOF RB-RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.Rotated(exp.LOF(metric="manhattan"))), 100), DF, train_ans, DF_test, ans)

RES["Polynom"] = exp.Solve(exp.PolynomSolver(metric="manhattan"), DF, train_ans, DF_test, ans)
RES["Polynom S"] = exp.Solve(exp.Sampler(exp.PolynomSolver(metric="manhattan"), 100), DF, train_ans, DF_test, ans)
RES["Polynom RB"] = exp.Solve(exp.Ensembler(exp.Rotated(exp.PolynomSolver(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["Polynom RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.PolynomSolver(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["Polynom RB-S"] = exp.Solve(exp.Sampler(exp.Rotated(exp.PolynomSolver(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["Polynom RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.PolynomSolver(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["Polynom RB-RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(exp.PolynomSolver(metric="manhattan"))), 100), DF, train_ans, DF_test, ans)
RES["Polynom RB-RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.Rotated(exp.PolynomSolver(metric="manhattan"))), 100), DF, train_ans, DF_test, ans)

RES["IterativeEnsemble"] = exp.Solve(exp.IterativeEnsemble(metric="manhattan"), DF, train_ans, DF_test, ans)

In [14]:
results["Arrythmia (train)"] = pd.Series({key: item[0] for key, item in RES.items()})
results["Arrythmia (test)"] = pd.Series({key: item[1] for key, item in RES.items()})
results.to_csv("FINAL_RESULTS.csv")
results

,Ionosphere (train),Ionosphere (test),Arrythmia (train),Arrythmia (test)
IsolationForest,0.923169,0.812883,0.779115,0.844961
IsolationForest RB,0.929172,0.805792,0.759569,0.786990
IsolationForest RB-RFW,0.932373,0.792432,0.763641,0.780923
IsolationForest RB-RFW-S,0.937175,0.811390,0.771423,0.815639
IsolationForest RB-S,0.926771,0.808031,0.783459,0.789350
IsolationForest S,0.927171,0.794298,0.762646,0.859454
IterativeEnsemble,0.910364,0.828034,0.775314,0.843950
LOF,0.960384,0.874832,0.796218,0.823391
LOF RB,0.903962,0.810942,0.767623,0.772835
LOF RB-RFW,0.919168,0.825123,0.766085,0.775531


# BREASTW

In [53]:
DF = pd.read_csv("breastcw.csv", sep=',', header=None)
for c in DF.columns:
    DF[c][DF[c].astype(str) == '?'] = -10
    DF[c] = DF[c].astype(float)
ans = (DF[10] == 4)
train_ans, ans = ans[:600], ans[600:]
del DF[10]

DF, DF_test = DF[:600], DF[600:]
DF

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,0,1,2,3,4,5,6,7,8,9
0,1000025.0,5.0,1.0,1.0,1.0,2.0,1.0,3.0,1.0,1.0
1,1002945.0,5.0,4.0,4.0,5.0,7.0,10.0,3.0,2.0,1.0
2,1015425.0,3.0,1.0,1.0,1.0,2.0,2.0,3.0,1.0,1.0
3,1016277.0,6.0,8.0,8.0,1.0,3.0,4.0,3.0,7.0,1.0
4,1017023.0,4.0,1.0,1.0,3.0,2.0,1.0,3.0,1.0,1.0
5,1017122.0,8.0,10.0,10.0,8.0,7.0,10.0,9.0,7.0,1.0
6,1018099.0,1.0,1.0,1.0,1.0,2.0,10.0,3.0,1.0,1.0
7,1018561.0,2.0,1.0,2.0,1.0,2.0,1.0,3.0,1.0,1.0
8,1033078.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,5.0
9,1033078.0,4.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0


In [15]:
scal = StandardScaler().fit(DF)
DF = scal.transform(DF)
DF_test = scal.transform(DF_test)
train_ans = train_ans.as_matrix()
ans = ans.as_matrix()

RES = {}

In [17]:
RES["IsolationForest"] = exp.Solve(IsolationForest(100, max_samples=1.0), DF, train_ans, DF_test, ans)
RES["IsolationForest S"] = exp.Solve(IsolationForest(100, max_samples=250), DF, train_ans, DF_test, ans)
RES["IsolationForest RB"] = exp.Solve(exp.Ensembler(exp.Rotated(IsolationForest(1, max_samples=1.0))), DF, train_ans, DF_test, ans)
RES["IsolationForest RB-S"] = exp.Solve(exp.Ensembler(exp.Rotated(IsolationForest(1, max_samples=250))), DF, train_ans, DF_test, ans)
RES["IsolationForest RB-RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(IsolationForest(1, max_samples=1.0)))), DF, train_ans, DF_test, ans)
RES["IsolationForest RB-RFW-S"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(IsolationForest(1, max_samples=250)))), DF, train_ans, DF_test, ans)

In [18]:
RES["LOF"] = exp.Solve(exp.LOF(metric="manhattan"), DF, train_ans, DF_test, ans)
RES["LOF S"] = exp.Solve(exp.Sampler(exp.LOF(metric="manhattan"), 100), DF, train_ans, DF_test, ans)
RES["LOF RB"] = exp.Solve(exp.Ensembler(exp.Rotated(exp.LOF(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["LOF RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.LOF(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["LOF RB-S"] = exp.Solve(exp.Sampler(exp.Rotated(exp.LOF(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["LOF RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.LOF(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["LOF RB-RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(exp.LOF(metric="manhattan"))), 100), DF, train_ans, DF_test, ans)
RES["LOF RB-RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.Rotated(exp.LOF(metric="manhattan"))), 100), DF, train_ans, DF_test, ans)

RES["Polynom"] = exp.Solve(exp.PolynomSolver(metric="manhattan"), DF, train_ans, DF_test, ans)
RES["Polynom S"] = exp.Solve(exp.Sampler(exp.PolynomSolver(metric="manhattan"), 100), DF, train_ans, DF_test, ans)
RES["Polynom RB"] = exp.Solve(exp.Ensembler(exp.Rotated(exp.PolynomSolver(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["Polynom RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.PolynomSolver(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["Polynom RB-S"] = exp.Solve(exp.Sampler(exp.Rotated(exp.PolynomSolver(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["Polynom RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.PolynomSolver(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["Polynom RB-RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(exp.PolynomSolver(metric="manhattan"))), 100), DF, train_ans, DF_test, ans)
RES["Polynom RB-RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.Rotated(exp.PolynomSolver(metric="manhattan"))), 100), DF, train_ans, DF_test, ans)

RES["IterativeEnsemble"] = exp.Solve(exp.IterativeEnsemble(metric="manhattan"), DF, train_ans, DF_test, ans)

In [19]:
results["Breastw (train)"] = pd.Series({key: item[0] for key, item in RES.items()})
results["Breastw (test)"] = pd.Series({key: item[1] for key, item in RES.items()})
results.to_csv("FINAL_RESULTS.csv")
results

,Ionosphere (train),Ionosphere (test),Arrythmia (train),Arrythmia (test),Breastw (train),Breastw (test)
IsolationForest,0.923169,0.812883,0.779115,0.844961,0.971459,0.994505
IsolationForest RB,0.929172,0.805792,0.759569,0.786990,0.945610,0.966422
IsolationForest RB-RFW,0.932373,0.792432,0.763641,0.780923,0.948792,0.971306
IsolationForest RB-RFW-S,0.937175,0.811390,0.771423,0.815639,0.942022,0.965812
IsolationForest RB-S,0.926771,0.808031,0.783459,0.789350,0.948457,0.967033
IsolationForest S,0.927171,0.794298,0.762646,0.859454,0.977978,0.995726
IterativeEnsemble,0.910364,0.828034,0.775314,0.843950,0.983529,0.987179
LOF,0.960384,0.874832,0.796218,0.823391,0.383457,0.347985
LOF RB,0.903962,0.810942,0.767623,0.772835,0.467129,0.423077
LOF RB-RFW,0.919168,0.825123,0.766085,0.775531,0.368900,0.335775


# PIMA

In [55]:
DF = pd.read_csv("pima.csv", sep=',', header=None)
ans = (DF[8] == 1)
train_ans, ans = ans[:576], ans[576:]
del DF[8]

DF, DF_test = DF[:576], DF[576:]
DF

,0,1,2,3,4,5,6,7
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
5,5,116,74,0,0,25.6,0.201,30
6,3,78,50,32,88,31.0,0.248,26
7,10,115,0,0,0,35.3,0.134,29
8,2,197,70,45,543,30.5,0.158,53
9,8,125,96,0,0,0.0,0.232,54


In [18]:
scal = StandardScaler().fit(DF)
DF = scal.transform(DF)
DF_test = scal.transform(DF_test)
train_ans = train_ans.as_matrix()
ans = ans.as_matrix()

RES = {}

In [22]:
RES["IsolationForest"] = exp.Solve(IsolationForest(100, max_samples=1.0), DF, train_ans, DF_test, ans)
RES["IsolationForest S"] = exp.Solve(IsolationForest(100, max_samples=250), DF, train_ans, DF_test, ans)
RES["IsolationForest RB"] = exp.Solve(exp.Ensembler(exp.Rotated(IsolationForest(1, max_samples=1.0))), DF, train_ans, DF_test, ans)
RES["IsolationForest RB-S"] = exp.Solve(exp.Ensembler(exp.Rotated(IsolationForest(1, max_samples=250))), DF, train_ans, DF_test, ans)
RES["IsolationForest RB-RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(IsolationForest(1, max_samples=1.0)))), DF, train_ans, DF_test, ans)
RES["IsolationForest RB-RFW-S"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(IsolationForest(1, max_samples=250)))), DF, train_ans, DF_test, ans)

In [23]:
RES["LOF"] = exp.Solve(exp.LOF(metric="manhattan"), DF, train_ans, DF_test, ans)
RES["LOF S"] = exp.Solve(exp.Sampler(exp.LOF(metric="manhattan"), 100), DF, train_ans, DF_test, ans)
RES["LOF RB"] = exp.Solve(exp.Ensembler(exp.Rotated(exp.LOF(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["LOF RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.LOF(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["LOF RB-S"] = exp.Solve(exp.Sampler(exp.Rotated(exp.LOF(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["LOF RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.LOF(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["LOF RB-RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(exp.LOF(metric="manhattan"))), 100), DF, train_ans, DF_test, ans)
RES["LOF RB-RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.Rotated(exp.LOF(metric="manhattan"))), 100), DF, train_ans, DF_test, ans)

RES["Polynom"] = exp.Solve(exp.PolynomSolver(metric="manhattan"), DF, train_ans, DF_test, ans)
RES["Polynom S"] = exp.Solve(exp.Sampler(exp.PolynomSolver(metric="manhattan"), 100), DF, train_ans, DF_test, ans)
RES["Polynom RB"] = exp.Solve(exp.Ensembler(exp.Rotated(exp.PolynomSolver(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["Polynom RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.PolynomSolver(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["Polynom RB-S"] = exp.Solve(exp.Sampler(exp.Rotated(exp.PolynomSolver(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["Polynom RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.PolynomSolver(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["Polynom RB-RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(exp.PolynomSolver(metric="manhattan"))), 100), DF, train_ans, DF_test, ans)
RES["Polynom RB-RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.Rotated(exp.PolynomSolver(metric="manhattan"))), 100), DF, train_ans, DF_test, ans)

RES["IterativeEnsemble"] = exp.Solve(exp.IterativeEnsemble(metric="manhattan"), DF, train_ans, DF_test, ans)

In [24]:
results["Pima (train)"] = pd.Series({key: item[0] for key, item in RES.items()})
results["Pima (test)"] = pd.Series({key: item[1] for key, item in RES.items()})
results.to_csv("FINAL_RESULTS.csv")
results

,Ionosphere (train),Ionosphere (test),Arrythmia (train),Arrythmia (test),Breastw (train),Breastw (test),Pima (train),Pima (test)
IsolationForest,0.923169,0.812883,0.779115,0.844961,0.971459,0.994505,0.679373,0.741569
IsolationForest RB,0.929172,0.805792,0.759569,0.786990,0.945610,0.966422,0.677623,0.711241
IsolationForest RB-RFW,0.932373,0.792432,0.763641,0.780923,0.948792,0.971306,0.652825,0.720492
IsolationForest RB-RFW-S,0.937175,0.811390,0.771423,0.815639,0.942022,0.965812,0.663647,0.712881
IsolationForest RB-S,0.926771,0.808031,0.783459,0.789350,0.948457,0.967033,0.670715,0.721429
IsolationForest S,0.927171,0.794298,0.762646,0.859454,0.977978,0.995726,0.667268,0.707963
IterativeEnsemble,0.910364,0.828034,0.775314,0.843950,0.983529,0.987179,0.689327,0.735363
LOF,0.960384,0.874832,0.796218,0.823391,0.383457,0.347985,0.617939,0.659602
LOF RB,0.903962,0.810942,0.767623,0.772835,0.467129,0.423077,0.619021,0.642155
LOF RB-RFW,0.919168,0.825123,0.766085,0.775531,0.368900,0.335775,0.625501,0.655621


# SATTELITE

In [57]:
DF = pd.read_csv("satellite.csv", sep=';', header=None)
train_ans = (DF[36] == 2) | (DF[36] == 4) | (DF[36] == 5)
del DF[36]
DF

DF_test = pd.read_csv("satellite_test.csv", sep=';', header=None)
ans = (DF_test[36] == 2) | (DF_test[36] == 4) | (DF_test[36] == 5)
del DF_test[36]
DF

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,92,115,120,94,84,102,106,79,84,102,...,134,104,88,121,128,100,84,107,113,87
1,84,102,106,79,84,102,102,83,80,102,...,128,100,84,107,113,87,84,99,104,79
2,84,102,102,83,80,102,102,79,84,94,...,113,87,84,99,104,79,84,99,104,79
3,80,102,102,79,84,94,102,79,80,94,...,104,79,84,99,104,79,84,103,104,79
4,84,94,102,79,80,94,98,76,80,102,...,104,79,84,103,104,79,79,107,109,87
5,80,94,98,76,80,102,102,79,76,102,...,104,79,79,107,109,87,79,107,109,87
6,76,102,106,83,76,102,106,87,80,98,...,113,87,79,103,104,83,79,103,104,79
7,76,102,106,87,80,98,106,79,76,94,...,104,83,79,103,104,79,79,95,100,79
8,76,89,98,76,76,94,98,76,76,98,...,96,75,75,91,96,71,79,87,93,71
9,76,94,98,76,76,98,102,72,76,94,...,96,71,79,87,93,71,79,87,93,67


In [21]:
scal = StandardScaler().fit(DF)
DF = scal.transform(DF)
DF_test = scal.transform(DF_test)
train_ans = train_ans.as_matrix()
ans = ans.as_matrix()

RES = {}

In [27]:
RES["IsolationForest"] = exp.Solve(IsolationForest(100, max_samples=1.0), DF, train_ans, DF_test, ans)
RES["IsolationForest S"] = exp.Solve(IsolationForest(100, max_samples=250), DF, train_ans, DF_test, ans)
RES["IsolationForest RB"] = exp.Solve(exp.Ensembler(exp.Rotated(IsolationForest(1, max_samples=1.0))), DF, train_ans, DF_test, ans)
RES["IsolationForest RB-S"] = exp.Solve(exp.Ensembler(exp.Rotated(IsolationForest(1, max_samples=250))), DF, train_ans, DF_test, ans)
RES["IsolationForest RB-RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(IsolationForest(1, max_samples=1.0)))), DF, train_ans, DF_test, ans)
RES["IsolationForest RB-RFW-S"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(IsolationForest(1, max_samples=250)))), DF, train_ans, DF_test, ans)

In [29]:
results["Sattelite (train)"] = pd.Series({key: item[0] for key, item in RES.items()})
results["Sattelite (test)"] = pd.Series({key: item[1] for key, item in RES.items()})
results.to_csv("FINAL_RESULTS.csv")
results

,Ionosphere (train),Ionosphere (test),Arrythmia (train),Arrythmia (test),Breastw (train),Breastw (test),Pima (train),Pima (test),Sattelite (train),Sattelite (test)
IsolationForest,0.923169,0.812883,0.779115,0.844961,0.971459,0.994505,0.679373,0.741569,0.767677,0.763803
IsolationForest RB,0.929172,0.805792,0.759569,0.786990,0.945610,0.966422,0.677623,0.711241,0.691966,0.683967
IsolationForest RB-RFW,0.932373,0.792432,0.763641,0.780923,0.948792,0.971306,0.652825,0.720492,0.716570,0.710095
IsolationForest RB-RFW-S,0.937175,0.811390,0.771423,0.815639,0.942022,0.965812,0.663647,0.712881,0.684552,0.676344
IsolationForest RB-S,0.926771,0.808031,0.783459,0.789350,0.948457,0.967033,0.670715,0.721429,0.691097,0.681655
IsolationForest S,0.927171,0.794298,0.762646,0.859454,0.977978,0.995726,0.667268,0.707963,0.694477,0.686605
IterativeEnsemble,0.910364,0.828034,0.775314,0.843950,0.983529,0.987179,0.689327,0.735363,0.651940,0.643473
LOF,0.960384,0.874832,0.796218,0.823391,0.383457,0.347985,0.617939,0.659602,0.556454,0.531530
LOF RB,0.903962,0.810942,0.767623,0.772835,0.467129,0.423077,0.619021,0.642155,0.546970,0.527943
LOF RB-RFW,0.919168,0.825123,0.766085,0.775531,0.368900,0.335775,0.625501,0.655621,0.547444,0.528477


# THYROID

In [61]:
DF = pd.read_csv("ann-train.csv", sep=' ', header=None)
train_ans = (DF[21] == 1) | (DF[21] == 2)
del DF[22]
del DF[23]
del DF[21]
for i in range(1, 16):
    del DF[i]

DF_test = pd.read_csv("ann-test.csv", sep=' ', header=None)
ans = (DF_test[21] == 1) | (DF_test[21] == 2)
del DF_test[21]
del DF_test[22]
del DF_test[23]
for i in range(1, 16):
    del DF_test[i]
DF

,0,16,17,18,19,20
0,0.73,0.00060,0.0150,0.120,0.082,0.146
1,0.24,0.00025,0.0300,0.143,0.133,0.108
2,0.47,0.00190,0.0240,0.102,0.131,0.078
3,0.64,0.00090,0.0170,0.077,0.090,0.085
4,0.23,0.00025,0.0260,0.139,0.090,0.153
5,0.69,0.00025,0.0160,0.086,0.070,0.123
6,0.85,0.00025,0.0230,0.128,0.104,0.121
7,0.48,0.00208,0.0200,0.086,0.078,0.110
8,0.67,0.00130,0.0240,0.087,0.109,0.080
9,0.76,0.00010,0.0290,0.124,0.128,0.097


In [24]:
scal = StandardScaler().fit(DF)
DF = scal.transform(DF)
DF_test = scal.transform(DF_test)
train_ans = train_ans.as_matrix()
ans = ans.as_matrix()

RES = {}

In [32]:
RES["IsolationForest"] = exp.Solve(IsolationForest(100, max_samples=1.0), DF, train_ans, DF_test, ans)
RES["IsolationForest S"] = exp.Solve(IsolationForest(100, max_samples=250), DF, train_ans, DF_test, ans)
RES["IsolationForest RB"] = exp.Solve(exp.Ensembler(exp.Rotated(IsolationForest(1, max_samples=1.0))), DF, train_ans, DF_test, ans)
RES["IsolationForest RB-S"] = exp.Solve(exp.Ensembler(exp.Rotated(IsolationForest(1, max_samples=250))), DF, train_ans, DF_test, ans)
RES["IsolationForest RB-RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(IsolationForest(1, max_samples=1.0)))), DF, train_ans, DF_test, ans)
RES["IsolationForest RB-RFW-S"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(IsolationForest(1, max_samples=250)))), DF, train_ans, DF_test, ans)

In [33]:
RES["LOF"] = exp.Solve(exp.LOF(metric="manhattan"), DF, train_ans, DF_test, ans)
RES["LOF S"] = exp.Solve(exp.Sampler(exp.LOF(metric="manhattan"), 100), DF, train_ans, DF_test, ans)
print('|')
RES["LOF RB"] = exp.Solve(exp.Ensembler(exp.Rotated(exp.LOF(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["LOF RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.LOF(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
print('|')
RES["LOF RB-S"] = exp.Solve(exp.Sampler(exp.Rotated(exp.LOF(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["LOF RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.LOF(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
print('|')
RES["LOF RB-RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(exp.LOF(metric="manhattan"))), 100), DF, train_ans, DF_test, ans)
RES["LOF RB-RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.Rotated(exp.LOF(metric="manhattan"))), 100), DF, train_ans, DF_test, ans)
print('|')
RES["Polynom"] = exp.Solve(exp.PolynomSolver(metric="manhattan"), DF, train_ans, DF_test, ans)
RES["Polynom S"] = exp.Solve(exp.Sampler(exp.PolynomSolver(metric="manhattan"), 100), DF, train_ans, DF_test, ans)
print('&')
RES["Polynom RB"] = exp.Solve(exp.Ensembler(exp.Rotated(exp.PolynomSolver(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["Polynom RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.PolynomSolver(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
print('&')
RES["Polynom RB-S"] = exp.Solve(exp.Sampler(exp.Rotated(exp.PolynomSolver(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["Polynom RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.PolynomSolver(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
print('&')
RES["Polynom RB-RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(exp.PolynomSolver(metric="manhattan"))), 100), DF, train_ans, DF_test, ans)
RES["Polynom RB-RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.Rotated(exp.PolynomSolver(metric="manhattan"))), 100), DF, train_ans, DF_test, ans)

RES["IterativeEnsemble"] = exp.Solve(exp.IterativeEnsemble(metric="manhattan"), DF, train_ans, DF_test, ans)

|
|
|
|
&
&
&


In [34]:
results["Thyroid (train)"] = pd.Series({key: item[0] for key, item in RES.items()})
results["Thyroid (test)"] = pd.Series({key: item[1] for key, item in RES.items()})
results.to_csv("FINAL_RESULTS.csv")
results

,Ionosphere (train),Ionosphere (test),Arrythmia (train),Arrythmia (test),Breastw (train),Breastw (test),Pima (train),Pima (test),Sattelite (train),Sattelite (test),Thyroid (train),Thyroid (test)
IsolationForest,0.923169,0.812883,0.779115,0.844961,0.971459,0.994505,0.679373,0.741569,0.767677,0.763803,0.836529,0.763257
IsolationForest RB,0.929172,0.805792,0.759569,0.786990,0.945610,0.966422,0.677623,0.711241,0.691966,0.683967,0.728778,0.631636
IsolationForest RB-RFW,0.932373,0.792432,0.763641,0.780923,0.948792,0.971306,0.652825,0.720492,0.716570,0.710095,0.733064,0.631037
IsolationForest RB-RFW-S,0.937175,0.811390,0.771423,0.815639,0.942022,0.965812,0.663647,0.712881,0.684552,0.676344,0.753857,0.650242
IsolationForest RB-S,0.926771,0.808031,0.783459,0.789350,0.948457,0.967033,0.670715,0.721429,0.691097,0.681655,0.722385,0.624009
IsolationForest S,0.927171,0.794298,0.762646,0.859454,0.977978,0.995726,0.667268,0.707963,0.694477,0.686605,0.827272,0.761987
IterativeEnsemble,0.910364,0.828034,0.775314,0.843950,0.983529,0.987179,0.689327,0.735363,0.651940,0.643473,0.767520,0.676407
LOF,0.960384,0.874832,0.796218,0.823391,0.383457,0.347985,0.617939,0.659602,0.556454,0.531530,0.702003,0.653125
LOF RB,0.903962,0.810942,0.767623,0.772835,0.467129,0.423077,0.619021,0.642155,0.546970,0.527943,0.702430,0.639157
LOF RB-RFW,0.919168,0.825123,0.766085,0.775531,0.368900,0.335775,0.625501,0.655621,0.547444,0.528477,0.761937,0.682899


# SHUTTLE

In [63]:
DF = pd.read_csv("shuttle.txt", sep=' ', header=None)
train_ans = (DF[9] == 2) | (DF[9] == 3) | (DF[9] == 5) | (DF[9] == 6) | (DF[9] == 7)
del DF[9]

DF_test = pd.read_csv("shuttle_test.txt", sep=' ', header=None)
ans = (DF_test[9] == 2) | (DF_test[9] == 3) | (DF_test[9] == 5) | (DF_test[9] == 6) | (DF_test[9] == 7)
del DF_test[9]

DF

,0,1,2,3,4,5,6,7,8
0,50,21,77,0,28,0,27,48,22
1,55,0,92,0,0,26,36,92,56
2,53,0,82,0,52,-5,29,30,2
3,37,0,76,0,28,18,40,48,8
4,37,0,79,0,34,-26,43,46,2
5,85,0,88,-4,6,1,3,83,80
6,56,0,81,0,-4,11,25,86,62
7,55,-1,95,-3,54,-4,40,41,2
8,53,8,77,0,28,0,23,48,24
9,37,0,101,-7,28,0,64,73,8


In [27]:
scal = StandardScaler().fit(DF)
DF = scal.transform(DF)
DF_test = scal.transform(DF_test)
train_ans = train_ans.as_matrix()
ans = ans.as_matrix()

RES = {}

In [37]:
RES["IsolationForest S"] = exp.Solve(IsolationForest(100, max_samples=250), DF, train_ans, DF_test, ans)
RES["IsolationForest RB-S"] = exp.Solve(exp.Ensembler(exp.Rotated(IsolationForest(1, max_samples=250))), DF, train_ans, DF_test, ans)
RES["IsolationForest RB-RFW-S"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(IsolationForest(1, max_samples=250)))), DF, train_ans, DF_test, ans)

In [38]:
RES["LOF S"] = exp.Solve(exp.Sampler(exp.LOF(metric="manhattan"), 100), DF, train_ans, DF_test, ans)
print('|')
print('|')
RES["LOF RB-S"] = exp.Solve(exp.Sampler(exp.Rotated(exp.LOF(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["LOF RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.LOF(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
print('|')
RES["LOF RB-RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.Rotated(exp.LOF(metric="manhattan"))), 100), DF, train_ans, DF_test, ans)
print('|')
RES["Polynom S"] = exp.Solve(exp.Sampler(exp.PolynomSolver(metric="manhattan"), 100), DF, train_ans, DF_test, ans)
print('&')
print('&')
RES["Polynom RB-S"] = exp.Solve(exp.Sampler(exp.Rotated(exp.PolynomSolver(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["Polynom RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.PolynomSolver(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
print('&')
RES["Polynom RB-RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.Rotated(exp.PolynomSolver(metric="manhattan"))), 100), DF, train_ans, DF_test, ans)

RES["IterativeEnsemble"] = exp.Solve(exp.IterativeEnsemble(metric="manhattan"), DF, train_ans, DF_test, ans)

|
|
|
|
&
&
&


In [39]:
results["Shuttle (train)"] = pd.Series({key: item[0] for key, item in RES.items()})
results["Shuttle (test)"] = pd.Series({key: item[1] for key, item in RES.items()})
results.to_csv("FINAL_RESULTS.csv")
results

,Ionosphere (train),Ionosphere (test),Arrythmia (train),Arrythmia (test),Breastw (train),Breastw (test),Pima (train),Pima (test),Sattelite (train),Sattelite (test),Thyroid (train),Thyroid (test),Shuttle (train),Shuttle (test)
IsolationForest,0.923169,0.812883,0.779115,0.844961,0.971459,0.994505,0.679373,0.741569,0.767677,0.763803,0.836529,0.763257,NaN,NaN
IsolationForest RB,0.929172,0.805792,0.759569,0.786990,0.945610,0.966422,0.677623,0.711241,0.691966,0.683967,0.728778,0.631636,NaN,NaN
IsolationForest RB-RFW,0.932373,0.792432,0.763641,0.780923,0.948792,0.971306,0.652825,0.720492,0.716570,0.710095,0.733064,0.631037,NaN,NaN
IsolationForest RB-RFW-S,0.937175,0.811390,0.771423,0.815639,0.942022,0.965812,0.663647,0.712881,0.684552,0.676344,0.753857,0.650242,0.989748,0.989358
IsolationForest RB-S,0.926771,0.808031,0.783459,0.789350,0.948457,0.967033,0.670715,0.721429,0.691097,0.681655,0.722385,0.624009,0.988106,0.987690
IsolationForest S,0.927171,0.794298,0.762646,0.859454,0.977978,0.995726,0.667268,0.707963,0.694477,0.686605,0.827272,0.761987,0.995780,0.995028
IterativeEnsemble,0.910364,0.828034,0.775314,0.843950,0.983529,0.987179,0.689327,0.735363,0.651940,0.643473,0.767520,0.676407,0.991202,0.990860
LOF,0.960384,0.874832,0.796218,0.823391,0.383457,0.347985,0.617939,0.659602,0.556454,0.531530,0.702003,0.653125,NaN,NaN
LOF RB,0.903962,0.810942,0.767623,0.772835,0.467129,0.423077,0.619021,0.642155,0.546970,0.527943,0.702430,0.639157,NaN,NaN
LOF RB-RFW,0.919168,0.825123,0.766085,0.775531,0.368900,0.335775,0.625501,0.655621,0.547444,0.528477,0.761937,0.682899,NaN,NaN


# ForestCover

In [65]:
test = sio.loadmat('cover.mat')
DF = pd.DataFrame(test['X'])
DF['y'] = test['y'][:, 0]
DF = shuffle(DF)

ans = DF['y'].astype(bool)
train_ans, ans = ans[:200000], ans[200000:]
del DF['y']

DF, DF_test = DF[:200000], DF[200000:]
DF

0.00960328336503


,0,1,2,3,4,5,6,7,8,9
154017,2738,273,12,90,3,1725,186,243,196,981
85154,2869,67,22,67,15,4910,236,190,75,95
83415,3058,63,4,390,89,5447,224,231,143,824
178391,2469,338,13,60,13,1123,191,221,169,799
145090,3063,126,6,450,-2,1438,231,236,138,1358
52425,2819,66,10,270,6,3250,230,219,121,2609
18465,3096,144,14,212,26,6112,239,237,124,2205
18049,2761,135,5,124,28,2673,229,238,143,5659
50439,2747,55,4,124,6,2310,222,231,145,1505
141727,2435,342,19,150,53,480,177,207,167,1052


In [30]:
scal = StandardScaler().fit(DF)
DF = scal.transform(DF)
DF_test = scal.transform(DF_test)
train_ans = train_ans.as_matrix()
ans = ans.as_matrix()

RES = {}

In [55]:
RES["IsolationForest S"] = exp.Solve(IsolationForest(100, max_samples=250), DF, train_ans, DF_test, ans)
RES["IsolationForest RB-S"] = exp.Solve(exp.Ensembler(exp.Rotated(IsolationForest(1, max_samples=250))), DF, train_ans, DF_test, ans)
RES["IsolationForest RB-RFW-S"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(IsolationForest(1, max_samples=250)))), DF, train_ans, DF_test, ans)

In [56]:
RES["LOF S"] = exp.Solve(exp.Sampler(exp.LOF(metric="chebyshev"), 100), DF, train_ans, DF_test, ans)
print('|')
print('|')
RES["LOF RB-S"] = exp.Solve(exp.Sampler(exp.Rotated(exp.LOF(metric="chebyshev")), 100), DF, train_ans, DF_test, ans)
RES["LOF RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.LOF(metric="chebyshev")), 100), DF, train_ans, DF_test, ans)
print('|')
RES["LOF RB-RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.Rotated(exp.LOF(metric="chebyshev"))), 100), DF, train_ans, DF_test, ans)
print('|')
RES["Polynom S"] = exp.Solve(exp.Sampler(exp.PolynomSolver(metric="chebyshev"), 100), DF, train_ans, DF_test, ans)
print('&')
print('&')
RES["Polynom RB-S"] = exp.Solve(exp.Sampler(exp.Rotated(exp.PolynomSolver(metric="chebyshev")), 100), DF, train_ans, DF_test, ans)
RES["Polynom RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.PolynomSolver(metric="chebyshev")), 100), DF, train_ans, DF_test, ans)
print('&')
RES["Polynom RB-RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.Rotated(exp.PolynomSolver(metric="chebyshev"))), 100), DF, train_ans, DF_test, ans)

RES["IterativeEnsemble"] = exp.Solve(exp.IterativeEnsemble(metric="chebyshev"), DF, train_ans, DF_test, ans)

|
|
|
|
&
&
&


In [57]:
results["ForestCover (train)"] = pd.Series({key: item[0] for key, item in RES.items()})
results["ForestCover (test)"] = pd.Series({key: item[1] for key, item in RES.items()})
results.to_csv("FINAL_RESULTS.csv")
results

,Ionosphere (train),Ionosphere (test),Arrythmia (train),Arrythmia (test),Breastw (train),Breastw (test),Pima (train),Pima (test),Sattelite (train),Sattelite (test),Thyroid (train),Thyroid (test),Shuttle (train),Shuttle (test),Mnist (train),Mnist (test),ForestCover (train),ForestCover (test)
IsolationForest,0.923169,0.812883,0.779115,0.844961,0.971459,0.994505,0.679373,0.741569,0.767677,0.763803,0.836529,0.763257,NaN,NaN,0.807134,0.811534,NaN,NaN
IsolationForest RB,0.929172,0.805792,0.759569,0.786990,0.945610,0.966422,0.677623,0.711241,0.691966,0.683967,0.728778,0.631636,NaN,NaN,0.828762,0.839441,NaN,NaN
IsolationForest RB-RFW,0.932373,0.792432,0.763641,0.780923,0.948792,0.971306,0.652825,0.720492,0.716570,0.710095,0.733064,0.631037,NaN,NaN,0.846853,0.853070,NaN,NaN
IsolationForest RB-RFW-S,0.937175,0.811390,0.771423,0.815639,0.942022,0.965812,0.663647,0.712881,0.684552,0.676344,0.753857,0.650242,0.989748,0.989358,0.848265,0.856885,0.913528,0.914820
IsolationForest RB-S,0.926771,0.808031,0.783459,0.789350,0.948457,0.967033,0.670715,0.721429,0.691097,0.681655,0.722385,0.624009,0.988106,0.987690,0.840312,0.841442,0.940392,0.941129
IsolationForest S,0.927171,0.794298,0.762646,0.859454,0.977978,0.995726,0.667268,0.707963,0.694477,0.686605,0.827272,0.761987,0.995780,0.995028,0.821065,0.811682,0.880860,0.882925
IterativeEnsemble,0.910364,0.828034,0.775314,0.843950,0.983529,0.987179,0.689327,0.735363,0.651940,0.643473,0.767520,0.676407,0.991202,0.990860,0.849638,0.854797,0.960014,0.960935
LOF,0.960384,0.874832,0.796218,0.823391,0.383457,0.347985,0.617939,0.659602,0.556454,0.531530,0.702003,0.653125,NaN,NaN,0.729816,0.742189,NaN,NaN
LOF RB,0.903962,0.810942,0.767623,0.772835,0.467129,0.423077,0.619021,0.642155,0.546970,0.527943,0.702430,0.639157,NaN,NaN,0.706885,0.735163,NaN,NaN
LOF RB-RFW,0.919168,0.825123,0.766085,0.775531,0.368900,0.335775,0.625501,0.655621,0.547444,0.528477,0.761937,0.682899,NaN,NaN,0.750591,0.756597,NaN,NaN


# MNIST

In [47]:
test = sio.loadmat('mnist.mat')
DF = pd.DataFrame(test['X'])
DF['y'] = test['y'][:, 0]
DF = shuffle(DF)

ans = DF['y'].astype(bool)
train_ans, ans = ans[:5000], ans[5000:]
del DF['y']

DF, DF_test = DF[:5000], DF[5000:]
DF

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
2067,-0.0036,66.195847,198.205963,0.0,-13.124617,-1.150100,0.0,-0.141633,179.249390,135.661163,...,104.055649,-0.107667,-4.469967,154.381409,-137.100632,-93.868584,-2.274633,-0.00065,-12.351267,0.0
6992,-0.0036,-73.804153,-53.794033,0.0,-13.124617,-1.150100,0.0,-0.141633,117.249382,-44.338833,...,-64.944351,-0.107667,-4.469967,-93.618584,-137.100632,-93.868584,-2.274633,-0.00065,-12.351267,0.0
972,-0.0036,-6.804153,199.205963,0.0,-13.124617,-1.150100,0.0,-0.141633,179.249390,-44.338833,...,188.055649,-0.107667,-4.469967,159.381409,-137.100632,-93.868584,-2.274633,-0.00065,241.648727,0.0
1091,-0.0036,-73.804153,-53.794033,0.0,-13.124617,-1.150100,0.0,-0.141633,179.249390,-44.338833,...,64.055649,-0.107667,-4.469967,129.381409,-137.100632,103.131416,-2.274633,-0.00065,242.648727,0.0
680,-0.0036,-73.804153,199.205963,0.0,-13.124617,63.849899,0.0,-0.141633,179.249390,-44.338833,...,-64.944351,-0.107667,-4.469967,34.381416,-137.100632,-93.868584,-2.274633,-0.00065,-12.351267,0.0
1619,-0.0036,-73.804153,199.205963,0.0,-13.124617,-1.150100,0.0,-0.141633,178.249390,-44.338833,...,-64.944351,-0.107667,-4.469967,158.381409,-137.100632,-93.868584,-2.274633,-0.00065,-12.351267,0.0
377,-0.0036,180.195847,-53.794033,0.0,-13.124617,-1.150100,0.0,-0.141633,68.249382,-44.338833,...,66.055649,-0.107667,-4.469967,-93.618584,-137.100632,-93.868584,-2.274633,-0.00065,-12.351267,0.0
970,-0.0036,179.195847,101.205963,0.0,-13.124617,-1.150100,0.0,-0.141633,179.249390,210.661163,...,134.055649,-0.107667,-4.469967,41.381416,-137.100632,-93.868584,-2.274633,-0.00065,-12.351267,0.0
2442,-0.0036,179.195847,-53.794033,0.0,-13.124617,-1.150100,0.0,-0.141633,180.249390,83.661163,...,189.055649,-0.107667,-4.469967,147.381409,-137.100632,-93.868584,-2.274633,-0.00065,-12.351267,0.0
5404,-0.0036,180.195847,199.205963,0.0,-13.124617,-1.150100,0.0,-0.141633,179.249390,-44.338833,...,-37.944351,-0.107667,-4.469967,45.381416,-137.100632,-93.868584,-2.274633,-0.00065,-12.351267,0.0


In [33]:
scal = StandardScaler().fit(DF)
DF = scal.transform(DF)
DF_test = scal.transform(DF_test)
train_ans = train_ans.as_matrix()
ans = ans.as_matrix()

RES = {}

In [42]:
RES["IsolationForest"] = exp.Solve(IsolationForest(100, max_samples=1.0), DF, train_ans, DF_test, ans)
RES["IsolationForest S"] = exp.Solve(IsolationForest(100, max_samples=250), DF, train_ans, DF_test, ans)
RES["IsolationForest RB"] = exp.Solve(exp.Ensembler(exp.Rotated(IsolationForest(1, max_samples=1.0))), DF, train_ans, DF_test, ans)
RES["IsolationForest RB-S"] = exp.Solve(exp.Ensembler(exp.Rotated(IsolationForest(1, max_samples=250))), DF, train_ans, DF_test, ans)
RES["IsolationForest RB-RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(IsolationForest(1, max_samples=1.0)))), DF, train_ans, DF_test, ans)
RES["IsolationForest RB-RFW-S"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(IsolationForest(1, max_samples=250)))), DF, train_ans, DF_test, ans)

In [43]:
RES["LOF"] = exp.Solve(exp.LOF(metric="manhattan"), DF, train_ans, DF_test, ans)
RES["LOF S"] = exp.Solve(exp.Sampler(exp.LOF(metric="manhattan"), 100), DF, train_ans, DF_test, ans)
print('|')
RES["LOF RB"] = exp.Solve(exp.Ensembler(exp.Rotated(exp.LOF(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["LOF RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.LOF(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
print('|')
RES["LOF RB-S"] = exp.Solve(exp.Sampler(exp.Rotated(exp.LOF(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["LOF RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.LOF(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
print('|')
RES["LOF RB-RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(exp.LOF(metric="manhattan"))), 100), DF, train_ans, DF_test, ans)
RES["LOF RB-RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.Rotated(exp.LOF(metric="manhattan"))), 100), DF, train_ans, DF_test, ans)
print('|')
RES["Polynom"] = exp.Solve(exp.PolynomSolver(metric="manhattan"), DF, train_ans, DF_test, ans)
RES["Polynom S"] = exp.Solve(exp.Sampler(exp.PolynomSolver(metric="manhattan"), 100), DF, train_ans, DF_test, ans)
print('&')
RES["Polynom RB"] = exp.Solve(exp.Ensembler(exp.Rotated(exp.PolynomSolver(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["Polynom RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.PolynomSolver(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
print('&')
RES["Polynom RB-S"] = exp.Solve(exp.Sampler(exp.Rotated(exp.PolynomSolver(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
RES["Polynom RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.PolynomSolver(metric="manhattan")), 100), DF, train_ans, DF_test, ans)
print('&')
RES["Polynom RB-RFW"] = exp.Solve(exp.Ensembler(exp.RndWeight(exp.Rotated(exp.PolynomSolver(metric="manhattan"))), 100), DF, train_ans, DF_test, ans)
RES["Polynom RB-RFW-S"] = exp.Solve(exp.Sampler(exp.RndWeight(exp.Rotated(exp.PolynomSolver(metric="manhattan"))), 100), DF, train_ans, DF_test, ans)

RES["IterativeEnsemble"] = exp.Solve(exp.IterativeEnsemble(metric="manhattan"), DF, train_ans, DF_test, ans)

|
|
|
|
&
&
&


In [44]:
results["Mnist (train)"] = pd.Series({key: item[0] for key, item in RES.items()})
results["Mnist (test)"] = pd.Series({key: item[1] for key, item in RES.items()})
results.to_csv("FINAL_RESULTS.csv")
results

,Ionosphere (train),Ionosphere (test),Arrythmia (train),Arrythmia (test),Breastw (train),Breastw (test),Pima (train),Pima (test),Sattelite (train),Sattelite (test),Thyroid (train),Thyroid (test),Shuttle (train),Shuttle (test),Mnist (train),Mnist (test)
IsolationForest,0.923169,0.812883,0.779115,0.844961,0.971459,0.994505,0.679373,0.741569,0.767677,0.763803,0.836529,0.763257,NaN,NaN,0.807134,0.811534
IsolationForest RB,0.929172,0.805792,0.759569,0.786990,0.945610,0.966422,0.677623,0.711241,0.691966,0.683967,0.728778,0.631636,NaN,NaN,0.828762,0.839441
IsolationForest RB-RFW,0.932373,0.792432,0.763641,0.780923,0.948792,0.971306,0.652825,0.720492,0.716570,0.710095,0.733064,0.631037,NaN,NaN,0.846853,0.853070
IsolationForest RB-RFW-S,0.937175,0.811390,0.771423,0.815639,0.942022,0.965812,0.663647,0.712881,0.684552,0.676344,0.753857,0.650242,0.989748,0.989358,0.848265,0.856885
IsolationForest RB-S,0.926771,0.808031,0.783459,0.789350,0.948457,0.967033,0.670715,0.721429,0.691097,0.681655,0.722385,0.624009,0.988106,0.987690,0.840312,0.841442
IsolationForest S,0.927171,0.794298,0.762646,0.859454,0.977978,0.995726,0.667268,0.707963,0.694477,0.686605,0.827272,0.761987,0.995780,0.995028,0.821065,0.811682
IterativeEnsemble,0.910364,0.828034,0.775314,0.843950,0.983529,0.987179,0.689327,0.735363,0.651940,0.643473,0.767520,0.676407,0.991202,0.990860,0.849638,0.854797
LOF,0.960384,0.874832,0.796218,0.823391,0.383457,0.347985,0.617939,0.659602,0.556454,0.531530,0.702003,0.653125,NaN,NaN,0.729816,0.742189
LOF RB,0.903962,0.810942,0.767623,0.772835,0.467129,0.423077,0.619021,0.642155,0.546970,0.527943,0.702430,0.639157,NaN,NaN,0.706885,0.735163
LOF RB-RFW,0.919168,0.825123,0.766085,0.775531,0.368900,0.335775,0.625501,0.655621,0.547444,0.528477,0.761937,0.682899,NaN,NaN,0.750591,0.756597
